# Laboratorio 00
### Curso MA4702 año 2021
### Profesores Iván Rapaport y José Soto 

Este es un laboratorio de prueba en el cual usaremos parte del tiempo para usar zoom en grupos, para que puedan trabajar en escribir y resolver modelos, y finalmente puedan entender el proceso de entrega.

Este laboratorio no tiene evaluación asociada pero si una entrega al final del mismo. 
Si bien la entrega no se evaluará, preocúpense de hacerla bien pues les puede servir de base para futuros laboratorios.
En particular, la tarea 1 que se mandará el día lunes hará uso de las funciones que cree hoy en el laboratorio.

Debe ingresar al zoom con su nombre y apellido tal como aparece en la lista de integrantes del curso (no se permiten nicknames o abreviaciones en el nombre durante los laboratorio).

La primera parte del laboratorio consiste en una descripción global del mismo y una prueba del sistema de asignación aleatoria de zoom en grupos.




## 1. Instrucciones.

* Complete la celda "2. Datos grupo".
* Lea el enunciado.
* Complete los ejercicios 1, 2, 3, 4 y 5 (hasta donde alcanze)
* (Recomendamos que uno de los integrantes del grupo mantenga la versión oficial de su grupo, el resto
  puede ejecutar celdas de prueba en sus propios Jupiter, cambiando el orden, etc)
* Antes de entregar, en el menú de Jupyter ejecute Kernel -> Restart and Run All. 
* Esto correrá todas las celdas del archivo en orden (puede tomar algo de tiempo)
* Guarde una copia del archivo .ipynb luego de esto. Eso es lo que debe entregar.
* Un representante del grupo debe subir el archivo .ipynb por el modulo de tareas de ucursos en la tarea: 
  Laboratorio de diagnostico.
* Plazo de entrega:  Miércoles 31 de marzo a las 18:00

## 2. Datos grupo

Grupo: 06

Integrantes:
- Arturo Lazcano
- Felipe Urrutia
- Francisco Maldonado
- Sebastián Cobaise

# 3 Preparacion

Antes de empezar a trabajar asegurese de tener los siguientes paquetes instalados
JuMP, Gurobi, CSV, Combinatorics
y luego llameslo con el comando ``using``

In [1]:
import Pkg

#Pkg.add("JuMP")
#Pkg.add("Gurobi")
#Pkg.add("CSV")
#Pkg.add("Combinatorics")
#Pkg.add("DataFrames")

# Nota: Luego de instalar estos paquetes
# comente todas las lineas anteriores para que no
# vuelvan a cargarse cuando ejecute el archivo nuevamente

using JuMP, Gurobi, CSV, Combinatorics, DataFrames


# 4. Lectura de grafos de un archivo.



En este laboratorio usaremos una estructura simple para grafos:

> Llamaremos ``N`` al número de vértices del grafo. Supondremos que sus vértices son elementos de ``1:N`` y que los pesos son valores enteros (esto es para evitar trabajar con decimales en los datos).
>
> Las aristas serán arreglos de objetos de 2 elementos ``[i,j]`` donde ``i<j``. Estarán en un arreglo ``E``.
>
> ``w`` será una diccionario tal que ``w[e]`` tiene el valor de w[e]
>
> (Note que si quiere acceder al peso de [1, 2] por ejemplo, debe escribir w[ [1, 2] ]

La siguiente funcion recibe un archivo .csv (tipo estándar para recibir datos) que corresponde a una tabla con 3 columnas de datos donde la primera fila es de la forma N 0 0 y el resto son cero o mas lineas con valores i j w

In [2]:
function leegrafo(archivocsv)
    M=Matrix(CSV.read(archivocsv, DataFrame, header=0))
    N=M[1,1]
    w=Dict{Array{Int64,1},Int64}()
    for i in 2:size(M,1)
        e=[M[i,1],M[i,2]]
        w[e]=M[i,3]
    end
    E=sort(collect(keys(w)))
return N,E,w
end

leegrafo (generic function with 1 method)

In [3]:
N,E,w=leegrafo("grafo1.csv")
@show(E)

E = [[1, 2], [1, 3], [1, 6], [1, 8], [2, 3], [2, 5], [3, 4], [3, 8], [4, 5], [4, 7], [4, 8], [5, 6], [5, 7], [6, 7], [7, 8]]


15-element Array{Array{Int64,1},1}:
 [1, 2]
 [1, 3]
 [1, 6]
 [1, 8]
 [2, 3]
 [2, 5]
 [3, 4]
 [3, 8]
 [4, 5]
 [4, 7]
 [4, 8]
 [5, 6]
 [5, 7]
 [6, 7]
 [7, 8]

In [4]:
for e in E
    @show(e,w[e])
end

e = [1, 2]
w[e] = 15
e = [1, 3]
w[e] = 15
e = [1, 6]
w[e] = 5
e = [1, 8]
w[e] = 5
e = [2, 3]
w[e] = 15
e = [2, 5]
w[e] = 5
e = [3, 4]
w[e] = 5
e = [3, 8]
w[e] = 5
e = [4, 5]
w[e] = 15
e = [4, 7]
w[e] = 5
e = [4, 8]
w[e] = 15
e = [5, 6]
w[e] = 15
e = [5, 7]
w[e] = 5
e = [6, 7]
w[e] = 15
e = [7, 8]
w[e] = 15


# 5. Formulaciones del MST

En este laboratorio investigaremos distintos modelos para el problema de encontrar un árbol generador de peso
mínimo (MST) en un grafo $G = (V,E)$ con pesos positivos $w\colon E \to \mathbb{R}^+$.
Como usted ya sabe, este problema se puede resolver mediante distintos algoritmos combinatoriales tales como el de Prim o el de Kruskal. Estos son en general mucho más eficientes de lo que se puede lograr con un solver de uso general para programación lineal mixta. Sin embargo los algoritmos combinatoriales no siempre se pueden modificar para manejar dificultades típicas como por ejemplo la inclusión de restricciones adicionales. Entender como resolver MST usando programación lineal mixta es útil como primer paso para resolver problemas más complejos.

Les pediremos que escriban y prueben 4 modelos diferentes del problema del MST. El primer modelo estará a medio hacer, el resto deberá crearlos desde cero. Al final del laboratorio, además, debe correr una parte adicional.

Para cada modelo (digamos que se llama NOMBRE) se le pedirá lo siguiente

1. Escribir una funcion modelo_nombre(N,E,w) que reciba valores N, E, w descritos anteriormente 
   y que devuelva un modelo en JuMP
   
2. Resolver el modelo en una instancia de prueba ubicado en el archivo "grafo1.csv"
   Para esto debe escribir en una celda de Jupyter instrucciones similares a la siguiente:
   ```julia
   N,E,w=leegrafo("grafo1.csv")
   @time mimodelo = modelo_nombre(N,E,w)
   @time optimize(modelo_nombre)
   # luego debe escribir instrucciones para que muestre en la pantalla el valor del MST encontrado
   # y el conjunto de aristas del MST encontrado 
   ```  
3. Nota: Puede probar sus modelos en los grafos de prueba "grafo1.csv" hasta "grafo9.csv". 
   En la tarea (que se publicará el día lunes) se les pedirá trabajar con estos grafos, con los modelos anteriores 
   y con nuevos modelos.


> Nota adicional.
> En algunos modelos necesitará trabajar con conjuntos potencia. 
> Básese en el siguiente ejemplo para conseguir lo que necesite
> (el comando powerset es parte del paquete Combinatorics que cargó al comienzo) 

In [5]:
?powerset

search: powerset GRB_INT_PAR_FLOWCOVERCUTS



```
powerset(a, min=0, max=length(a))
```

Generate all subsets of an indexable object `a` including the empty set, with cardinality bounded by `min` and `max`. Because the number of subsets can be very large, this function returns an iterator object. Use `collect(powerset(a, min, max))` to get an array of all subsets.


In [6]:
# Ejemplo: imprimir todos los subconjuntos de 1:10 de tamaños 3 a 5 cuya suma de elementos esté entre 15 y 16
 for X in powerset(1:10, 3, 5)
    if sum(X)>=15 && sum(X)<=16
        println(X)
    end
end

[1, 4, 10]
[1, 5, 9]
[1, 5, 10]
[1, 6, 8]
[1, 6, 9]
[1, 7, 8]
[2, 3, 10]
[2, 4, 9]
[2, 4, 10]
[2, 5, 8]
[2, 5, 9]
[2, 6, 7]
[2, 6, 8]
[3, 4, 8]
[3, 4, 9]
[3, 5, 7]
[3, 5, 8]
[3, 6, 7]
[4, 5, 6]
[4, 5, 7]
[1, 2, 3, 9]
[1, 2, 3, 10]
[1, 2, 4, 8]
[1, 2, 4, 9]
[1, 2, 5, 7]
[1, 2, 5, 8]
[1, 2, 6, 7]
[1, 3, 4, 7]
[1, 3, 4, 8]
[1, 3, 5, 6]
[1, 3, 5, 7]
[1, 4, 5, 6]
[2, 3, 4, 6]
[2, 3, 4, 7]
[2, 3, 5, 6]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 6]


# 6. Ejercicios.

## Ejercicio 1. Formulacion del MST como PLE por corte

Recuerde que un subgrafo $H = (V, F)$ de $G=(V,E)$ es conexo si y solo si la siguiente condición de corte se cumple:
   
>   $\delta_F(S) \geq 1$, para todo $S\subseteq V, S\neq \emptyset, S\neq V$
   
Recuerde además que  los árboles generadores de $G$ son exactamente los subgrafos $H = (V, F)$ conexos con $|F| = n−1$ aristas.

Usando la caracterización anterior, el modelo (M1) siguiente resulta ser un modelo exacto para MST

$$
\begin{align*}
(\text{M1}) \quad \min \sum_{e\in E}&w_e x_e\\
 \sum_{e \in \delta_E(S)} x_e= x(\delta_E(S))&\geq 1, \qquad \text{para todo $S\subseteq V, S\neq \emptyset, S\neq V$.}\\
x(E) &= n-1\\
0\leq x_e  &\leq 1, \qquad \text{para todo $e\in E$}\\
x_e &\in \mathbb{Z}
\end{align*}
$$

### Parte 1.a. 
Escribir una funcion ``modelo_cortes(N,E,w)`` que reciba valores N, E, w descritos anteriormente y que devuelva un modelo en JuMP

In [7]:
function modelo_cortes(N,E,w)
    # 1. Crear modelo, asignar Gurobi como el optimizador
    mimodelo = Model()
    set_optimizer(mimodelo, Gurobi.Optimizer)
    
    set_optimizer_attributes(mimodelo, "outputflag"=> 0)
    
    # 2. Declarar variables de decisión (recomendamos usar este formato)
    @variable(mimodelo, 0<=x[i in 1:N,j in 1:N; [i,j] in E]<=1, Int)
    
    #3. Función objetivo
    @objective(mimodelo, Min, sum(x[e[1],e[2]]*w[e] for e in E))

    # 3. Escribir restricciones (llenar)
    for S in powerset(1:N, 1, N-1)
        @constraint(mimodelo, sum(x[e[1], e[2]] for e in E if ((e[1] in S) && !(e[2] in S)) || ((e[2] in S) && !(e[1] in S))) >= 1)
    end
    # no olvidar ninguna restriccion  
    @constraint(mimodelo, sum(x[e[1], e[2]] for e in E) == N-1)
    
    return mimodelo
end


modelo_cortes (generic function with 1 method)

### Parte 1.b 
Resolver MST, usando el modelo, en una instancia de prueba ubicado en el archivo "grafo1.csv". 

In [8]:
# Lea el grafo y cree el modelo, mostrando el tiempo de ejecución
N,E,w=leegrafo("grafo1.csv")
println("Grafo con $N nodos y $(size(E,1)) aristas")
@time mimodelo = modelo_cortes(N,E,w);

Grafo con 8 nodos y 15 aristas
Academic license - for non-commercial use only - expires 2021-05-20
  3.442344 seconds (14.00 M allocations: 722.976 MiB, 7.54% gc time)


In [9]:
# Optimice el modelo, mostrando el tiempo de ejecución
@time optimize!(mimodelo);

  7.048259 seconds (32.17 M allocations: 1.614 GiB, 6.93% gc time)


In [10]:
# Muestre el valor objetivo (objective_value)
objective_value(mimodelo)
value.(mimodelo[:x])

JuMP.Containers.SparseAxisArray{Float64,2,Tuple{Int64,Int64}} with 15 entries:
  [1, 6]  =  1.0
  [2, 3]  =  -0.0
  [5, 6]  =  -0.0
  [2, 5]  =  1.0
  [4, 7]  =  1.0
  [4, 5]  =  -0.0
  [1, 3]  =  -0.0
  [4, 8]  =  -0.0
  [7, 8]  =  -0.0
  [1, 8]  =  1.0
  [6, 7]  =  -0.0
  [1, 2]  =  -0.0
  [3, 8]  =  1.0
  [3, 4]  =  1.0
  [5, 7]  =  1.0

In [11]:
# Mostrar aristas de valor 1
[e for e in E if value(mimodelo[:x][e[1],e[2]]) ≈ 1]

7-element Array{Array{Int64,1},1}:
 [1, 6]
 [1, 8]
 [2, 5]
 [3, 4]
 [3, 8]
 [4, 7]
 [5, 7]

## Ejercicio 2. Formulacion del MST como PLE por ciclos

Recuerde que un subgrafo $H = (V, F)$ de $G=(V,E)$ es aciclico si y solo si la siguiente condición de eliminación de  ciclos se cumple:
   
>   $|F[S]|\leq |S|-1$, para todo $S\subseteq V, S\neq \emptyset, S\neq V$
   
Recuerde además que  los árboles generadores de $G$ son exactamente los subgrafos $H = (V, F)$ acíclicos con $|F| = n−1$ aristas.

Usando la caracterización anterior, el modelo (M2) siguiente resulta ser un modelo exacto para MST

$$
\begin{align*}
(\text{M2}) \quad \min \sum_{e\in E}&w_e x_e\\
x(E[S])&\leq |S|-1, \qquad \text{para todo $S\subseteq V, S\neq \emptyset, S\neq V$.}\\
x(E) &= n-1\\
0\leq x_e  &\leq 1, \qquad \text{para todo $e\in E$}\\
x_e &\in \mathbb{Z}
\end{align*}
$$

### Parte 2.a. 
Escribir una funcion ``modelo_ciclos(N,E,w)`` que reciba valores N, E, w descritos anteriormente y que devuelva un modelo en JuMP para (M2)

In [12]:
function modelo_ciclos(N,E,w)
    mimodelo = Model()
    set_optimizer(mimodelo, Gurobi.Optimizer)
    
    set_optimizer_attributes(mimodelo, "outputflag"=> 0)
    
    @variable(mimodelo, 0<= x[i in 1:N, j in 1:N; [i, j] in E]<=1, Int)
    
    @objective(mimodelo, Min, sum(x[e[1], e[2]] * w[e] for e in E))
    
    for S in powerset(1:N, 1, N-1)
        @constraint(mimodelo, sum(x[e[1], e[2]] for e in E if (e[1] in S && e[2] in S)) <= length(S) - 1)
    end
    @constraint(mimodelo, sum(x[e[1], e[2]] for e in E) == N-1)
        
    return mimodelo
end

modelo_ciclos (generic function with 1 method)

### Parte 2.b 
Resolver MST, usando el modelo, en una instancia de prueba ubicado en el archivo "grafo1.csv". 

In [13]:
# Lea el grafo y cree el modelo, mostrando el tiempo de ejecución
N,E,w=leegrafo("grafo1.csv")
@time mimodelo=modelo_ciclos(N, E, w);

Academic license - for non-commercial use only - expires 2021-05-20
  0.287022 seconds (690.93 k allocations: 36.788 MiB, 9.96% gc time)


In [14]:
# Optimice el modelo, mostrando el tiempo de ejecución
@time optimize!(mimodelo);

  0.410227 seconds (1.33 M allocations: 68.770 MiB, 20.32% gc time)


In [15]:
# Muestre el valor objetivo (objective_value)
objective_value(mimodelo)

35.0

In [16]:
# Mostrar aristas de valor 1
[e for e in E if value(mimodelo[:x][e[1], e[2]]) ≈ 1]

7-element Array{Array{Int64,1},1}:
 [1, 6]
 [1, 8]
 [2, 5]
 [3, 4]
 [3, 8]
 [4, 7]
 [5, 7]

## Ejercicio 3. Formulacion del MST como PLE por arborescencias

Sea $G' = (V,A)$ el digrafo obtenido al reemplazar cada arista $ij$ de E por dos arcos antiparalelos $(i, j); (j, i) \in A$.

Un subgrafo $H=(V,F)$ de $G=(V,E)$ es un árbol generador si y solo si podemos dirigir este árbol a una arborescencia enraizada en un nodo fijo $r\in V$. En otras palabras
1. Debemos poder elegir exactamente una dirección para cada e en E
2. Cada nodo $v\in V$ tiene exactamente un arco entrante (que viene de su padre), excepto $r\in V$ que no tiene padres. Esto dice que hay exactamente n-1 nodos.

Podemos escribir esto como un programa lineal entero extendido (es decir con variables adicionales $\{y_a\}_{a\in A}$ como sigue (aqui $r\in V$ es un vértice fijo que se elige antes)


$$\begin{align*}
(\text{M3}) \quad \min \sum_{e\in E}&w_e x_e\\
x(E) &= n-1\\
y_{i,j}+y_{j,i}&= x_e, \qquad\text{para todo $e=ij\in E$}\\
y(\delta^-(i))&= 1, \qquad \text{para todo $i\in V-r$}\\
y(\delta^-(r))&= 0,\\
y_a&\geq 0, \qquad \text{para todo $(i,j)\in A$}\\
0\leq x_e  &\leq 1, \qquad \text{para todo $e\in E$}\\
x_e &\in \mathbb{Z}, \qquad \text{para todo $e\in E$}\\
y_a &\in \mathbb{Z}, \qquad \text{para todo $a\in A$}\\
\end{align*}
$$

### Parte 3.a. 
Escribir una funcion ``modelo_arborescencia(N,E,w)`` que reciba valores N, E, w descritos anteriormente y que devuelva un modelo en JuMP para (M3)

In [17]:
function modelo_arborescencia(N,E,w)
    mimodelo = Model(Gurobi.Optimizer)
    
    set_optimizer_attributes(mimodelo, "outputflag"=> 0)
    
    # nota: puede crear el conjunto A primero, use r=1
    # De preferencia acá indexe las variables "y" por pares ordenados 
    # y[i in ..., j in...; [i,j] in ...], no por y[a in A], pues de esta ultima manera debe usar y[[i,j]] para trabajar.
    
    
    
    A = copy(E)
    append!(A, [[e[2], e[1]] for e in E])
    
    @variable(mimodelo, 0<= x[i in 1:N, j in 1:N; [i, j] in E]<=1, Int)
    
    @variable(mimodelo, 0<= y[i in 1:N, j in 1:N; [i, j] in A], Int)
    
    @objective(mimodelo, Min, sum(x[e[1], e[2]] * w[e] for e in E))
    
    @constraint(mimodelo, sum(x[e[1], e[2]] for e in E) == N-1)
    
    for e in E
        @constraint(mimodelo, y[e[1], e[2]] + y[e[2], e[1]] == x[e[1], e[2]])
    end
    
    for i in 2:N
        @constraint(mimodelo, sum(y[j, i] for j in 1:N if [j, i] in A) >= 1)
    end
    
    @constraint(mimodelo, sum(y[j, 1] for j in 1:N if [j, 1] in A) == 0)
    
    # nota: No necesita usar for para crear restricciones simples, puede indexarlas directamente
    # @constraint(mimodelo, [i in 1:N], restriccion para i)
    return mimodelo
end

modelo_arborescencia (generic function with 1 method)

### Parte 3.b 
Resolver MST, usando el modelo, en una instancia de prueba ubicado en el archivo "grafo1.csv". 

In [18]:
# Lea el grafo y cree el modelo, mostrando el tiempo de ejecución
N, E, w = leegrafo("grafo1.csv")
@time mimodelo = modelo_arborescencia(N,E,w)

Academic license - for non-commercial use only - expires 2021-05-20
  0.292633 seconds (787.55 k allocations: 41.635 MiB)


A JuMP Model
Minimization problem with:
Variables: 45
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 17 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 45 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 15 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 45 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

In [19]:
# Optimice el modelo, mostrando el tiempo de ejecución
@time optimize!(mimodelo)

  0.002194 seconds (1.31 k allocations: 140.375 KiB)


In [20]:
# Muestre el valor objetivo (objective_value)
@time objective_value(mimodelo)

  0.000018 seconds (6 allocations: 128 bytes)


35.0

In [21]:
# Mostrar aristas de valor 1
[e for e in E if value(mimodelo[:x][e[1], e[2]]) ≈ 1]

7-element Array{Array{Int64,1},1}:
 [1, 6]
 [1, 8]
 [2, 5]
 [3, 4]
 [3, 8]
 [4, 7]
 [5, 7]

## Ejercicio 4. Formulacion del MST por componentes

Defina el conjunto 
$S=\{(i,j,k)\colon i\neq j, j\neq k, k\neq i, \{i,j\}\in E\}$

Si $T$ es un árbol, y definimos $x_{\{i,j\}}$ como la indicatriz del evento $\{i,j\}\in T$ y $z_{i,j,k}$ como la indicatriz del evento $\{i,j\}$ es arista de $T$ y $k$ está en la componente de $j$ en el grafo $T-ij$, entonces se puede probar que este punto es factible en el PLE que listaremos a continuación. Usando esto se puede probar que el siguiente modelo es exacto:


$$\begin{align*}
(\text{M4}) \quad \min \sum_{e\in E}&w_e x_e\\
x(E) &= n-1\\
z_{i,j,k}+z_{j,i,k}&= x_e, \qquad\text{para todo $e=ij\in E$}\\
x_{i,k} + \sum_{j\in V\colon (i,j,k)\in S}z_{i,j,k}&=1, \qquad\text{para todo $ik\in E$}\\
z_{i,j,k}&\geq 0, \qquad \text{para todo $(i,j,k)\in S$}\\
0\leq x_e  &\leq 1, \qquad \text{para todo $e\in E$}\\
x_e &\in \mathbb{Z}, \qquad \text{para todo $e\in E$}\\
z_{i,j,k} &\in \mathbb{Z}, \qquad \text{para todo $(i,j,k)\in S$}\\
\end{align*}
$$

### Parte 4.a. 
Escribir una funcion ``modelo_componentes(N,E,w)`` que reciba valores N, E, w descritos anteriormente y que devuelva un modelo en JuMP para (M3)

In [22]:
function modelo_componentes(N,E,w)
    mimodelo = Model(Gurobi.Optimizer)
    set_optimizer_attributes(mimodelo, "outputflag"=> 0)

    S = [ [i, j, k] for i in 1:N for j in 1:N for k in 1:N if (i != j && j !=k && k != i && ([i, j] in E || [j, i] in E))]
    
    @variable(mimodelo, 0<= x[i in 1:N, j in 1:N; [i, j] in E]<=1, Int)
    @variable(mimodelo, 0<= z[i in 1:N, j in 1:N, k in 1:N; [i, j, k] in S], Int)
    
    @objective(mimodelo, Min, sum(x[e[1], e[2]] * w[e] for e in E))
    
    @constraint(mimodelo, sum(x[e[1], e[2]] for e in E) == N-1)
    
    for e in E
        for k in 1:N
            if [e[1], e[2], k] in S
                @constraint(mimodelo, z[e[1], e[2], k] + z[e[2], e[1], k] == x[e[1], e[2]])
            end
        end
    end
    
    for e in E
        @constraint(mimodelo, x[e[1], e[2]] + sum(z[e[1], j, e[2]] for j in 1:N if [e[1], j, e[2]] in S) == 1)
    end
    
    # nota: puede crear el conjunto S primero
    
    return mimodelo
end

modelo_componentes (generic function with 1 method)

### Parte 4.b 
Resolver MST, usando el modelo, en una instancia de prueba ubicado en el archivo "grafo1.csv". 

In [23]:
# Lea el grafo y cree el modelo, mostrando el tiempo de ejecución
N, E, w = leegrafo("grafo1.csv")
@time mimodelo4 = modelo_componentes(N, E, w)

Academic license - for non-commercial use only - expires 2021-05-20
  0.617744 seconds (1.90 M allocations: 97.833 MiB, 5.16% gc time)


A JuMP Model
Minimization problem with:
Variables: 195
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 106 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 195 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 15 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 195 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, z

In [24]:
# Optimice el modelo, mostrando el tiempo de ejecución
optimize!(mimodelo4);

In [25]:
# Muestre el valor objetivo (objective_value)
objective_value(mimodelo4)

35.0

In [26]:
# Mostrar aristas de valor 1
value.(mimodelo4[:x])

JuMP.Containers.SparseAxisArray{Float64,2,Tuple{Int64,Int64}} with 15 entries:
  [1, 6]  =  1.0
  [2, 3]  =  -0.0
  [5, 6]  =  -0.0
  [2, 5]  =  1.0
  [4, 7]  =  1.0
  [4, 5]  =  -0.0
  [1, 3]  =  -0.0
  [4, 8]  =  -0.0
  [7, 8]  =  -0.0
  [1, 8]  =  1.0
  [6, 7]  =  0.0
  [1, 2]  =  -0.0
  [3, 8]  =  1.0
  [3, 4]  =  1.0
  [5, 7]  =  1.0

## Ejercicio 5. Pequeña prueba de la calidad de cada formulación

Cree una función ``Prueba(N,e,w)`` que reciba valores N,E,w y realice lo siguiente

1. Para cada modelo resuelva el problema asociado 2 veces: una vez con todas las variables enteras y la segunda vez quitando la restricción de integralidad (puede usar el comando unset_integer(nombremodelo[:nombrevariable]))

La única salida que se pide, es que al final de todas las ejecuciones, no entremedio del texto que manda Gurobi, se
imprima en la pantalla 4 lineas, una con cada modelo. Como sigue:

> Modelo (NOMBRE):  Valor del PLE = (valor_objetivo_PLE). Valor del PL = (valor_objetivo_PL)

2. Ejecute su función en los grafos "grafo1.csv" al "grafo5.csv".


In [27]:
function Prueba(N, E, w)
    output = ""
    for modelo in [modelo_cortes, modelo_ciclos, modelo_arborescencia, modelo_componentes]
        mimodelo = modelo(N, E, w);
        optimize!(mimodelo);
        valor_objetivo_PLE = objective_value(mimodelo);
        mimodelo = modelo(N, E, w);
        for t in mimodelo[:x]
            unset_integer(t)
        end
        if modelo == modelo_arborescencia
            for t in mimodelo[:y]
                unset_integer(t)
            end
        end
        if modelo == modelo_componentes
            for t in mimodelo[:z]
                unset_integer(t)
            end
        end
        optimize!(mimodelo);
        valor_objetivo_PL = objective_value(mimodelo);
        output = output * "Modelo ($modelo): Valor del PLE = $valor_objetivo_PLE . Valor del PL = $valor_objetivo_PL" * "\n"
    end
    println(output)
end

Prueba (generic function with 1 method)

In [28]:
for i in 1:5   
    println("---grafo$i.csv---")
    N, e, w = leegrafo("grafo$i.csv")
    Prueba(N, e, w)
end

---grafo1.csv---
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Modelo (modelo_cortes): Valor del PLE = 35.0 . Valor del PL = 35.0
Modelo (modelo_ciclos): Valor del PLE = 35.0 . Valor del PL = 35.0
Modelo (modelo_arborescencia): Valor del PLE = 35.0 . Valor del PL = 35.0
Modelo (modelo_componentes): Valor del PLE = 35.0 . Valor del PL = 35.0

---grafo2.csv---
Academic license - for non-commercial use only - expires 2021-05-20
Academic license - for non-commercial use only - expires 2021-05-20
Acade